# Importar Dados

Dados de empréstimos dos acervos do sistema de bibliotecas da UFRN. 

In [3]:
import pandas as pd
import os

## Coleta e Organização dos Dados

## Unificar em um único Dataframe todos os dados pertinentes para a análise. 

### Dados de empréstimos

In [4]:
arquivos = os.listdir('./Dia_1-Importando_dados/Datasets/dados_emprestimos/')
arquivos

['emprestimos-20101.csv',
 'emprestimos-20102.csv',
 'emprestimos-20111.csv',
 'emprestimos-20112.csv',
 'emprestimos-20121.csv',
 'emprestimos-20122.csv',
 'emprestimos-20131.csv',
 'emprestimos-20132.csv',
 'emprestimos-20141.csv',
 'emprestimos-20142.csv',
 'emprestimos-20151.csv',
 'emprestimos-20152.csv',
 'emprestimos-20161.csv',
 'emprestimos-20162.csv',
 'emprestimos-20171.csv',
 'emprestimos-20172.csv',
 'emprestimos-20181.csv',
 'emprestimos-20182.csv',
 'emprestimos-20191.csv',
 'emprestimos-20192.csv',
 'emprestimos-20201.csv']

Após listar os arquivos a serem importados, eles serão unificados em um único DataFrame através do método contat do pandas.

In [19]:
tabela_x = pd.DataFrame()

for arquivo in arquivos:
    tabela_1 = pd.read_csv(f'./Dia_1-Importando_dados/Datasets/dados_emprestimos/{arquivo}')
    tabela_x = pd.concat([tabela_x, tabela_1])    

In [20]:
tabela_x.reset_index(inplace=True)

Mesclando com os dados do acervo, para entender, por exemplo, de qual biblioteca era o material emprestado ou a qual tema ele se referia. A relação se dá pela coluna de código de barras de cada material. 

### Dados do acervo

Para utilização do pandas na leitura de arquivo .parquet é necessário instalar a biblioteca pyarrow

In [24]:
!pip install pyarrow

     -------------------------------------- 21.5/21.5 MB 588.6 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
acervo = pd.read_parquet('./Dia_1-Importando_dados/Datasets/dados_exemplares.parquet', engine='pyarrow')
acervo

,id_exemplar,codigo_barras,colecao,biblioteca,status_material,localizacao,registro_sistema
index,,,,,,,
0,5,L000003,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,694,1
1,4,L000002,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,688,1
2,3,L000001,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,638,1
3,7,L000114,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,616,5
5,10,L000041,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,657,15
...,...,...,...,...,...,...,...
568580,1353472,2021011150,Acervo Circulante,Biblioteca Setorial do Núcleo de Educação da I...,REGULAR,951,268231
568581,1353473,2019013454,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,997,268233
568582,1353474,2019012811,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,987,268234


## Identificando dados nulos e vazios

In [32]:
# isna
print(tabela_x.isna().sum())

index                         0
id_emprestimo                 0
codigo_barras                 0
data_renovacao          1285720
data_emprestimo               0
data_devolucao             6471
matricula_ou_siape         3170
tipo_vinculo_usuario          0
dtype: int64


In [33]:
# isnull
print(tabela_x.isnull().sum())

index                         0
id_emprestimo                 0
codigo_barras                 0
data_renovacao          1285720
data_emprestimo               0
data_devolucao             6471
matricula_ou_siape         3170
tipo_vinculo_usuario          0
dtype: int64


In [34]:
# isna
print(acervo.isna().sum())

id_exemplar         0
codigo_barras       0
colecao             0
biblioteca          0
status_material     0
localizacao         0
registro_sistema    0
dtype: int64


In [35]:
# isnull
print(acervo.isnull().sum())

id_exemplar         0
codigo_barras       0
colecao             0
biblioteca          0
status_material     0
localizacao         0
registro_sistema    0
dtype: int64


As colunas que apresentam dados nulos não afetarão a junção dos dataframes nesse momento.

## Removendo dados duplicados

Não poderemos trabalhar com registros com dados do código de barras duplicados no acervo pois serão utilizados como chaves para o merge.

In [41]:
print(acervo['codigo_barras'].nunique())
print(len(acervo))

545049
546237


Para tabela dos empréstimos não poderão ter registros duplicados com mesmo número na coluna id_emprestimo.

In [42]:
print(tabela_x['id_emprestimo'].nunique())
print(len(tabela_x))

2139267
2258018


Eliminando os registros duplicados com o método drop_duplicates

In [54]:
acervo_df = acervo.drop_duplicates(subset='codigo_barras', keep='first')
emprestimos_df = tabela_x.drop_duplicates(subset='id_emprestimo', keep='first')

In [55]:
print(emprestimos_df['id_emprestimo'].nunique())
print(len(emprestimos_df))

2139267
2139267


In [57]:
print(acervo_df['codigo_barras'].nunique())
print(len(acervo_df))

545049
545049


### Merge entre dos dois DataFrames (Acervo e Empréstimos)

In [59]:
df_final = pd.merge(emprestimos_df, acervo_df, on = "codigo_barras")

In [60]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1955743 entries, 0 to 1955742
Data columns (total 14 columns):
 #   Column                Dtype  
---  ------                -----  
 0   index                 int64  
 1   id_emprestimo         int64  
 2   codigo_barras         object 
 3   data_renovacao        object 
 4   data_emprestimo       object 
 5   data_devolucao        object 
 6   matricula_ou_siape    float64
 7   tipo_vinculo_usuario  object 
 8   id_exemplar           int64  
 9   colecao               object 
 10  biblioteca            object 
 11  status_material       object 
 12  localizacao           int64  
 13  registro_sistema      int64  
dtypes: float64(1), int64(5), object(8)
memory usage: 223.8+ MB


In [61]:
# isna
print(df_final.isna().sum())

index                         0
id_emprestimo                 0
codigo_barras                 0
data_renovacao          1096082
data_emprestimo               0
data_devolucao             6138
matricula_ou_siape         2705
tipo_vinculo_usuario          0
id_exemplar                   0
colecao                       0
biblioteca                    0
status_material               0
localizacao                   0
registro_sistema              0
dtype: int64


In [62]:
# isnull
print(df_final.isnull().sum())

index                         0
id_emprestimo                 0
codigo_barras                 0
data_renovacao          1096082
data_emprestimo               0
data_devolucao             6138
matricula_ou_siape         2705
tipo_vinculo_usuario          0
id_exemplar                   0
colecao                       0
biblioteca                    0
status_material               0
localizacao                   0
registro_sistema              0
dtype: int64


DataFrame preparado para iniciar as análises